Install necessary libraries required to run the code in Google Colab

In [ ]:
!pip install crewai crewai_tools langchain openai langchain_community -- quiet

Get access to together AI free api key - https://api.together.xyz/models  & Serper API key - https://serper.dev/api-key

## Financial Researcher - Crewai

In [ ]:
from crewai_tools import SerperDevTool
import os
from crewai import LLM,Agent, Task, Crew
from langchain.llms import OpenAI
from google.colab import userdata
# Set your OpenAI key

os.environ["SERPER_API_KEY"] = userdata.get('serper_api')

llm = LLM(model="together_ai/deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
          api_key=userdata.get('together_ai'),
          base_url="https://api.together.xyz/v1"
        )

research_agent = Agent(
    llm = llm,
    role="Senior Financial Researcher for {company}",
    goal="Research the company, news and potential for {company}",
    backstory="""You're a seasoned financial researcher with a talent for finding
    the most relevant information about {company}.
    Known for your ability to find the most relevant
    information and present it in a clear and concise manner.""",
    tools=[SerperDevTool()],
    verbose=True  # Enable logging for debugging
)
analyst_agent= Agent(
    llm = llm,
    role="Market Analyst and Report writer focused on {company}",
    goal=""" Analyze company {company} and create a comprehensive, well-structured report
    that presents insights in a clear and engaging way""",
    backstory=""" You're a meticulous, skilled analyst with a background in financial analysis
    and company research. You have a talent for identifying patterns and extracting
    meaningful insights from research data, then communicating
    those insights through well crafted reports""",
    tools=[SerperDevTool()],
    verbose=True  # Enable logging for debugging
)

research_task = Task(
    description="""Conduct thorough research on company {company}. Focus on:
    1. Current company status and health
    2. Historical company performance
    3. Major challenges and opportunities
    4. Recent news and events
    5. Future outlook and potential developments

    Make sure to organize your findings in a structured format with clear sections.""",
    expected_output=""" A comprehensive research document with well-organized sections covering
    all the requested aspects of {company}. Include specific facts, figures,
    and examples where relevant.""",
    agent=research_agent
)

analysis_task = Task(
    description="""Analyze the research findings and create a comprehensive report on {company}.
    Your report should:
    1. Begin with an executive summary
    2. Include all key information from the research
    3. Provide insightful analysis of trends and patterns
    4. Offer a market outlook for company, noting that this should not be used for trading decisions
    5. Be formatted in a professional, easy-to-read style with clear headings""",
    expected_output="""  A polished, professional report on {company} that presents the research
    findings with added analysis and insights. The report should be well-structured
    with an executive summary, main sections, and conclusion.""",
    agent=analyst_agent
)

# Execute the crew
crew = Crew(
    agents=[research_agent,analyst_agent],
    tasks=[research_task,analysis_task],
    verbose=True
)


In [ ]:
inputs = {
        'company': 'Apple'
    }

result = crew.kickoff(inputs=inputs)

In [ ]:
print(result.raw)